# OLX case study: Quality assessment

A pragmatic solution to the exercise is presented. It consists of three notebooks that discuss the main ideas, its limitations and possible extensions. This is the first notebook that describes a "quick and dirty" approach to image quality assessment on the basis of a blurriness score.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from utils import DataImport, BlurScore

## Data import

Let us start by importing and exploring the available data. For experimentation purposes, a class to load and contain the data has been created. One can pass an optional function that guesses a class label from the image file name. It is not clear whether or not that possibility was intended by the task. We use it nevertheless. After all, it is more than plausible to assume that one has indeed an indication of what photos in a classified ad contain. It is remarked that possibly existing Exif metadata, for example related to the camera orientation, has not been checked during the import.

In [ ]:
data_dir = '../data' # data directory

guess_label = lambda file_name:'clock' if '_' not in file_name else 'other' # label from file name

data = DataImport(data_dir, guess_label)
data.print_summary()

Roughly two thirds seem to contain a clock. Let us have a look at some of the images. The first two rows of the plot below show examples with clocks. The last row contains other objects, most of which seem to be technical (car) components. After screening the data, we conclude that guessing labels makes good sense. A variety of clocks (analog, digital, ...) is identified this way. Most often the front side is shown, yet other orientations are possible.

Only sometimes an image that is supposed to feature a clock, actually shows its original packaging only. Very seldomly a related item such as the invoice or a store shelf is found. One might see this as a weak form of label noise. There are both amateur photographs as well as commercial pictures with white backgrounds. Some images contain watermarks.

In [ ]:
plot_size = (3, 4)

plot_ids = np.concatenate([
    np.random.choice(np.where(np.asarray(data.labels)=='clock')[0],
                     size=(plot_size[0]-1)*plot_size[1], replace=False),
    np.random.choice(np.where(np.asarray(data.labels)!='clock')[0],
                     size=plot_size[1], replace=False)
])

fig, axes = plt.subplots(nrows=plot_size[0], ncols=plot_size[1], figsize=(9,6))
for idx, ax in enumerate(axes.ravel().tolist()):
    image = data.images[plot_ids[idx]]
    label = data.labels[plot_ids[idx]]
    ax.imshow(image)
    ax.set_title(label)
    ax.set(xticks=[], yticks=[])
fig.tight_layout()

## Blurriness score

Very certainly there are many possibilities for image quality assessment. They might include statements about the sharpness, contrast, noise for example. Moreover, one might try to distinguish amateur from professional photos or to detect watermarks. We, however, focus on a very simple blurriness score. It is defined as the reciprocal of the standard deviation of the response under an edge-detecting Laplacian filter.

In [ ]:
blur_score = BlurScore()
blurriness = blur_score.fit_evaluate(data.images)

The above cell determines the blur scores of all available images and normalizes by the maximal value. While the scores are therefore between zero and one, it might well exceed that range for new images. A histogram of the blurriness score over all images is shown below. It can be seen that the large majority of the example images has a small blur only. The smaller the blur, the better the quality.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(blurriness, bins=50, range=(0, 1), histtype='stepfilled', alpha=0.7)
ax.set(xlim=(0, 1), xlabel='blurriness', ylabel='counts')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In order to get an intuitive idea of the definition of blurriness, we have a look at images of different blurriness. The first and last row of the following figure show the four images with the smallest and highest blur, respectively.
Four examples with a "medium" score can be seen in the middle row. It is concluded that the used bluriness score in fact allows for a meaningful first quality assessment.

In [ ]:
plot_size = (3, 4)

sorted_ids = np.argsort(blurriness) # sorted according to increasing blurriness
quotient, remainder = divmod(len(blurriness), np.prod(plot_size))
plot_ids = sorted_ids[0:len(blurriness)-remainder:quotient] # equally thinned
plot_ids[:plot_size[1]] = sorted_ids[:plot_size[1]] # lowest blurriness
plot_ids[-plot_size[1]:] = sorted_ids[-plot_size[1]:] # hightest blurriness

fig, axes = plt.subplots(nrows=plot_size[0], ncols=plot_size[1], figsize=(9, 6))
for idx, ax in enumerate(axes.ravel().tolist()):
    image = data.images[plot_ids[idx]]
    label = 'blurriness : {:.2f}'.format(blurriness[plot_ids[idx]])
    ax.imshow(image)
    ax.set_title(label)
    ax.set(xticks=[], yticks=[])
fig.tight_layout()